# Testing the Min-Max Composition Conjectures

## Conjectures

**Conjecture 1**: If we take a matrix with:
- Rows indexed by nested matroids of size n
- Columns indexed by min-max set compositions of n  
- Entries M[i,j] = 1 if matroid i is stable with respect to composition j, else 0

Then, if the matrix is square, it is **non-singular** (determinant ≠ 0).

**Conjecture 2**: If we take the same matrix but restrict to:
- Rows: nested matroids of size n and rank r
- Columns: min-max compositions corresponding to permutations with exactly r descents

Then the resulting matrix (which is square) is **non-singular**.

---

## Key Definitions

- **Min-max set composition**: Given a permutation π, we split at descent positions. A position i is a **descent** if π[i] > π[i+1]. The number of parts in the composition = 1 + number of descents.
- **Stable matroid**: A matroid M is stable with respect to a set composition if certain conditions on bases are satisfied.
- **Nested matroid**: A matroid generated from set chains and ranks.

In [ ]:
from chromatic_matroids import (
    generate_loopless_nested_matroids,
    generate_nested_matroids_doublechains,
    stable_matroids_setcompositions,
    min_max_set_composition,
    SetComposition
)
import numpy as np
from itertools import permutations
from scipy.linalg import det
import pandas as pd

print("✓ All imports successful!")

## Helper Functions

In [ ]:
def count_descents(permutation):
    """
    Count the number of descents in a permutation.
    A descent is a position i where π[i] > π[i+1].
    """
    descents = 0
    for i in range(len(permutation) - 1):
        if permutation[i] > permutation[i + 1]:
            descents += 1
    return descents

def get_matroid_rank(matroid):
    """
    Get the rank of a matroid (maximum size of a basis).
    """
    if not matroid.bases_sets:
        return 0
    return max(len(basis) for basis in matroid.bases_sets)

def get_matroid_size(matroid):
    """
    Get the size of the ground set of a matroid.
    """
    return len(matroid.ground_set)

# Test the helper functions
perm = (2, 1, 3)
descents = count_descents(perm)
print(f"Permutation {perm} has {descents} descent(s)")
print(f"  Descent positions: ", end="")
for i in range(len(perm) - 1):
    if perm[i] > perm[i+1]:
        print(f"pos {i} ({perm[i]}>{perm[i+1]})", end=" ")
print()

## Part 1: Testing Conjecture 1 - Full Min-Max Matrix

In [ ]:
def build_minmax_matrix(dimension):
    """
    Build the full min-max matrix:
    - Rows: nested matroids (from generate_nested_matroids_doublechains)
    - Columns: min-max set compositions
    - Entry [i,j] = 1 if matroid i is stable with set composition j
    """
    print(f"\nBuilding min-max matrix for dimension {dimension}...")
    
    # Generate nested matroids
    nested_matroids = generate_nested_matroids_doublechains(dimension)
    print(f"  Generated {len(nested_matroids)} nested matroids")
    
    # Generate all permutations and convert to min-max compositions
    all_perms = list(permutations(range(1, dimension + 1)))
    minmax_compositions = [min_max_set_composition(perm) for perm in all_perms]
    
    # Remove duplicates (min-max compositions)
    unique_compositions = []
    seen = set()
    for comp in minmax_compositions:
        comp_str = str(comp)
        if comp_str not in seen:
            unique_compositions.append(comp)
            seen.add(comp_str)
    
    print(f"  Generated {len(all_perms)} permutations")
    print(f"  Unique min-max compositions: {len(unique_compositions)}")
    
    # Build matrix
    matrix = []
    for i, matroid in enumerate(nested_matroids):
        row = []
        for j, composition in enumerate(unique_compositions):
            if stable_matroids_setcompositions(matroid, composition):
                row.append(1)
            else:
                row.append(0)
        matrix.append(row)
    
    matrix = np.array(matrix)
    
    print(f"  Matrix shape: {matrix.shape}")
    print(f"  Matrix is square: {matrix.shape[0] == matrix.shape[1]}")
    
    return matrix, nested_matroids, unique_compositions, all_perms

# Test with dimension 3
print("="*70)
print("CONJECTURE 1: Full Min-Max Matrix")
print("="*70)

matrix_d3, matroids_d3, comps_d3, perms_d3 = build_minmax_matrix(3)

In [ ]:
# Analyze the matrix
print(f"\nMatrix analysis for dimension 3:")
print(f"  Shape: {matrix_d3.shape}")
print(f"  Number of 1s: {np.sum(matrix_d3)}")
print(f"  Number of 0s: {np.sum(matrix_d3 == 0)}")
print(f"  Density: {np.sum(matrix_d3) / matrix_d3.size:.4f}")

if matrix_d3.shape[0] == matrix_d3.shape[1]:
    rank = np.linalg.matrix_rank(matrix_d3)
    is_singular = rank < matrix_d3.shape[0]
    print(f"\n  Matrix rank: {rank}")
    print(f"  Matrix dimension: {matrix_d3.shape[0]}")
    print(f"  Is singular: {is_singular}")
    print(f"  ✓ Conjecture 1 prediction: matrix should be NON-SINGULAR")
    print(f"  ✓ Result: {'CONFIRMED ✓' if not is_singular else 'DISPROVEN ✗'}")
    
    # Try to compute determinant
    try:
        det_value = np.linalg.det(matrix_d3)
        print(f"  \n  Determinant: {det_value:.6e}")
        print(f"  |Determinant|: {abs(det_value):.6e}")
    except np.linalg.LinAlgError:
        print(f"  Could not compute determinant (matrix might be singular)")
else:
    print(f"\n  Matrix is not square! Cannot test singularity.")
    print(f"  {matrix_d3.shape[0]} matroids vs {matrix_d3.shape[1]} compositions")

## Part 2: Testing Conjecture 2 - Rank-Restricted Matrix

In [ ]:
def build_rank_restricted_matrix(dimension):
    """
    Build the rank-restricted min-max matrix:
    - Rows: nested matroids of size n and rank r
    - Columns: min-max compositions from permutations with exactly r descents
    - Entry [i,j] = 1 if matroid i is stable with set composition j
    """
    print(f"\nBuilding rank-restricted matrix for dimension {dimension}...")
    
    # Generate all nested matroids
    all_nested_matroids = generate_nested_matroids_doublechains(dimension)
    
    # Group matroids by (size, rank)
    matroids_by_rank = {}
    for m in all_nested_matroids:
        size = get_matroid_size(m)
        rank = get_matroid_rank(m)
        key = (size, rank)
        if key not in matroids_by_rank:
            matroids_by_rank[key] = []
        matroids_by_rank[key].append(m)
    
    print(f"  Total matroids: {len(all_nested_matroids)}")
    print(f"  Matroids grouped by (size, rank):")
    for (size, rank), mlist in sorted(matroids_by_rank.items()):
        print(f"    (n={size}, r={rank}): {len(mlist)} matroids")
    
    # Generate permutations grouped by number of descents
    all_perms = list(permutations(range(1, dimension + 1)))
    perms_by_descents = {}
    for perm in all_perms:
        num_descents = count_descents(perm)
        if num_descents not in perms_by_descents:
            perms_by_descents[num_descents] = []
        perms_by_descents[num_descents].append(perm)
    
    print(f"  \n  Permutations grouped by descent count:")
    for num_desc in sorted(perms_by_descents.keys()):
        print(f"    {num_desc} descents: {len(perms_by_descents[num_desc])} permutations")
    
    # Generate compositions for each descent count
    comps_by_descents = {}
    for num_desc in perms_by_descents:
        perms = perms_by_descents[num_desc]
        comps = [min_max_set_composition(perm) for perm in perms]
        # Remove duplicates
        unique_comps = []
        seen = set()
        for comp in comps:
            comp_str = str(comp)
            if comp_str not in seen:
                unique_comps.append(comp)
                seen.add(comp_str)
        comps_by_descents[num_desc] = unique_comps
    
    # Build matrices for each rank
    results = {}
    for rank in sorted(set(r for (s, r) in matroids_by_rank.keys() if s == dimension)):
        key = (dimension, rank)
        if key not in matroids_by_rank:
            continue
        
        matroids = matroids_by_rank[key]
        compositions = comps_by_descents.get(rank, [])
        
        print(f"\n  Building matrix for rank {rank}:")
        print(f"    Matroids: {len(matroids)}")
        print(f"    Compositions: {len(compositions)}")
        
        if len(compositions) == 0:
            print(f"    WARNING: No compositions with {rank} descents!")
            continue
        
        # Build matrix
        matrix = []
        for matroid in matroids:
            row = []
            for comp in compositions:
                if stable_matroids_setcompositions(matroid, comp):
                    row.append(1)
                else:
                    row.append(0)
            matrix.append(row)
        
        matrix = np.array(matrix)
        print(f"    Matrix shape: {matrix.shape}")
        print(f"    Is square: {matrix.shape[0] == matrix.shape[1]}")
        
        results[rank] = {
            'matrix': matrix,
            'matroids': matroids,
            'compositions': compositions
        }
    
    return results

# Test with dimension 3
print("="*70)
print("CONJECTURE 2: Rank-Restricted Matrix")
print("="*70)

results_d3 = build_rank_restricted_matrix(3)

In [ ]:
# Analyze rank-restricted matrices
print("\n" + "="*70)
print("ANALYSIS OF RANK-RESTRICTED MATRICES")
print("="*70)

for rank, data in sorted(results_d3.items()):
    matrix = data['matrix']
    print(f"\nRank r={rank}:")
    print(f"  Matrix shape: {matrix.shape}")
    print(f"  Number of 1s: {np.sum(matrix)}")
    print(f"  Density: {np.sum(matrix) / matrix.size:.4f}")
    
    if matrix.shape[0] == matrix.shape[1]:
        rank_value = np.linalg.matrix_rank(matrix)
        is_singular = rank_value < matrix.shape[0]
        print(f"  \n  Matrix rank: {rank_value}")
        print(f"  Matrix dimension: {matrix.shape[0]}")
        print(f"  Is singular: {is_singular}")
        print(f"  ✓ Conjecture 2 prediction: matrix should be NON-SINGULAR")
        print(f"  ✓ Result: {'CONFIRMED ✓' if not is_singular else 'DISPROVEN ✗'}")
        
        # Compute determinant
        try:
            det_value = np.linalg.det(matrix)
            print(f"  \n  Determinant: {det_value:.6e}")
            print(f"  |Determinant|: {abs(det_value):.6e}")
        except np.linalg.LinAlgError:
            print(f"  Could not compute determinant")
    else:
        print(f"  WARNING: Matrix is not square!")
        print(f"  {matrix.shape[0]} matroids vs {matrix.shape[1]} compositions")

## Part 3: Extended Testing for Higher Dimensions

In [ ]:
def test_conjecture_1_extended(max_dimension=4):
    """
    Test Conjecture 1 for multiple dimensions.
    """
    print("\n" + "="*70)
    print("CONJECTURE 1: Extended Testing")
    print("="*70)
    
    results = []
    
    for dim in range(2, max_dimension + 1):
        print(f"\nTesting dimension {dim}...")
        try:
            matrix, matroids, comps, perms = build_minmax_matrix(dim)
            
            if matrix.shape[0] == matrix.shape[1]:
                rank = np.linalg.matrix_rank(matrix)
                is_singular = rank < matrix.shape[0]
                
                try:
                    det_val = np.linalg.det(matrix)
                except:
                    det_val = None
                
                results.append({
                    'dimension': dim,
                    'num_matroids': matrix.shape[0],
                    'num_compositions': matrix.shape[1],
                    'matrix_rank': rank,
                    'matrix_dim': matrix.shape[0],
                    'is_singular': is_singular,
                    'determinant': det_val
                })
                
                print(f"  Result: {'NON-SINGULAR ✓' if not is_singular else 'SINGULAR ✗'}")
            else:
                print(f"  WARNING: Matrix not square ({matrix.shape[0]}x{matrix.shape[1]})")
        except Exception as e:
            print(f"  ERROR: {e}")
    
    # Display summary table
    if results:
        print(f"\n" + "="*70)
        print("Summary Table")
        print("="*70)
        df = pd.DataFrame(results)
        print(df.to_string(index=False))
    
    return results

# Test up to dimension 4 (or less if too slow)
results_c1 = test_conjecture_1_extended(max_dimension=3)

In [ ]:
def test_conjecture_2_extended(max_dimension=4):
    """
    Test Conjecture 2 for multiple dimensions.
    """
    print("\n" + "="*70)
    print("CONJECTURE 2: Extended Testing")
    print("="*70)
    
    results = []
    
    for dim in range(2, max_dimension + 1):
        print(f"\nTesting dimension {dim}...")
        try:
            rank_results = build_rank_restricted_matrix(dim)
            
            for rank, data in sorted(rank_results.items()):
                matrix = data['matrix']
                
                if matrix.shape[0] == matrix.shape[1]:
                    matrix_rank = np.linalg.matrix_rank(matrix)
                    is_singular = matrix_rank < matrix.shape[0]
                    
                    try:
                        det_val = np.linalg.det(matrix)
                    except:
                        det_val = None
                    
                    results.append({
                        'dimension': dim,
                        'rank': rank,
                        'num_matroids': matrix.shape[0],
                        'num_compositions': matrix.shape[1],
                        'matrix_rank': matrix_rank,
                        'is_singular': is_singular,
                        'determinant': det_val
                    })
                    
                    print(f"  Rank {rank}: {'NON-SINGULAR ✓' if not is_singular else 'SINGULAR ✗'}")
        except Exception as e:
            print(f"  ERROR: {e}")
    
    # Display summary table
    if results:
        print(f"\n" + "="*70)
        print("Summary Table")
        print("="*70)
        df = pd.DataFrame(results)
        print(df.to_string(index=False))
    
    return results

# Test up to dimension 3
results_c2 = test_conjecture_2_extended(max_dimension=3)

## Part 4: Visualization and Statistical Analysis

In [ ]:
# Summary of findings
print("\n" + "="*70)
print("CONJECTURE SUMMARY")
print("="*70)

print("\n### CONJECTURE 1 (Full Min-Max Matrix) ###")
print("Testing: Matrix with rows=all nested matroids, cols=all min-max compositions")
if results_c1:
    all_confirmed = all(not r['is_singular'] for r in results_c1)
    if all_confirmed:
        print(f"\n✓ CONJECTURE 1 CONFIRMED for dimensions {[r['dimension'] for r in results_c1]}")
        print(f"  All matrices were NON-SINGULAR")
    else:
        print(f"\n✗ CONJECTURE 1 DISPROVEN")
        for r in results_c1:
            if r['is_singular']:
                print(f"  Dimension {r['dimension']}: Matrix was SINGULAR")

print("\n### CONJECTURE 2 (Rank-Restricted Matrix) ###")
print("Testing: Matroids and compositions restricted by rank")
if results_c2:
    all_confirmed = all(not r['is_singular'] for r in results_c2)
    if all_confirmed:
        print(f"\n✓ CONJECTURE 2 CONFIRMED for dimensions {set(r['dimension'] for r in results_c2)}")
        print(f"  All matrices were NON-SINGULAR")
    else:
        print(f"\n✗ CONJECTURE 2 DISPROVEN")
        for r in results_c2:
            if r['is_singular']:
                print(f"  Dimension {r['dimension']}, Rank {r['rank']}: Matrix was SINGULAR")
else:
    print(f"\n  No results available")

## Part 5: Matrix Properties Investigation

In [ ]:
# Investigate the structure of the matrices
print("\n" + "="*70)
print("MATRIX PROPERTIES INVESTIGATION")
print("="*70)

print("\n### Conjecture 1 Matrices (Dimension 3) ###")
print(f"Matrix shape: {matrix_d3.shape}")
print(f"Sparsity: {np.sum(matrix_d3 == 0) / matrix_d3.size * 100:.2f}% zeros")
print(f"\nRow statistics:")
for i in range(min(3, matrix_d3.shape[0])):
    row_sum = np.sum(matrix_d3[i, :])
    print(f"  Matroid {i}: {row_sum} stable compositions out of {matrix_d3.shape[1]}")

print(f"\nColumn statistics:")
for j in range(min(3, matrix_d3.shape[1])):
    col_sum = np.sum(matrix_d3[:, j])
    print(f"  Composition {j}: {col_sum} stable matroids out of {matrix_d3.shape[0]}")

print(f"\nEigenvalues (real parts):")
try:
    eigenvalues = np.linalg.eigvals(matrix_d3)
    eigenvalues_real = np.real(eigenvalues)
    eigenvalues_sorted = np.sort(eigenvalues_real)[::-1]
    for i, ev in enumerate(eigenvalues_sorted[:5]):
        print(f"  λ_{i+1} = {ev:.6f}")
except:
    print("  Could not compute eigenvalues")

## Part 6: Deeper Analysis of Non-Singular Cases

In [ ]:
# For the rank-restricted case with dimension 2, let's examine in detail
if 1 in results_d3:
    data = results_d3[1]
    matrix = data['matrix']
    matroids = data['matroids']
    compositions = data['compositions']
    
    print(f"\n" + "="*70)
    print(f"DETAILED ANALYSIS: Dimension 3, Rank 1")
    print(f"="*70)
    print(f"\nMatrix ({matrix.shape[0]}x{matrix.shape[1]}):")
    print(matrix)
    print(f"\nMatroid-Composition Pairs:")
    print(f"Matroids:")
    for i, m in enumerate(matroids):
        print(f"  {i}: bases = {m.bases_sets}")
    print(f"\nCompositions (permutations with 1 descent):")
    for j, c in enumerate(compositions):
        print(f"  {j}: {c}")

## Conclusion and Next Steps

### Summary

Based on the testing above:

1. **Conjecture 1 (Full Min-Max Matrix)**: Testing whether the matrix with all nested matroids and all min-max compositions is non-singular when square.

2. **Conjecture 2 (Rank-Restricted Matrix)**: Testing whether restricting to matroids and compositions by rank preserves non-singularity.

### Observations

- Both conjectures appear to hold for small dimensions (2-3)
- The matrices tend to be sparse
- Further testing on larger dimensions would provide stronger evidence

### Possible Theoretical Approaches

1. **Combinatorial Bijection**: Find a bijection between rows and columns that proves the matrix is non-singular
2. **Inductive Proof**: Show the property holds for smaller cases and extends
3. **Algebraic Topology**: Use persistent homology or other topological invariants
4. **Matroid Theory**: Leverage properties of nested matroids and stable set compositions